In [ ]:
import keras
from keras.datasets import cifar10
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten, ZeroPadding2D
from keras.layers import Conv2D, MaxPooling2D
from keras.callbacks import TensorBoard
import os

In [ ]:
num_classes = 10

In [ ]:
# The data, shuffled and split between train and test sets:
(x_train, y_train), (x_test, y_test) = cifar10.load_data()
print('x_train shape:', x_train.shape)
print('y_train shape:', y_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

In [ ]:
# Convert class vectors to binary class matrices.
x_train = x_train / 255
x_test = x_test / 255
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

In [ ]:
# VGG 16
model = Sequential()

# Block 1
model.add(ZeroPadding2D((1,1),input_shape=x_train.shape[1:]))
model.add(Conv2D(64, (3, 3), activation='relu', name='block1_conv1'))
model.add(ZeroPadding2D((1,1)))
model.add(Conv2D(64, (3, 3), activation='relu', name='block1_conv2'))
model.add(MaxPooling2D((2,2), strides=(2,2), name='block1_pool'))

# Block 2
model.add(ZeroPadding2D((1,1)))
model.add(Conv2D(128, (3, 3), activation='relu', name='block2_conv1'))
model.add(ZeroPadding2D((1,1)))
model.add(Conv2D(128, (3, 3), activation='relu', name='block2_conv2'))
model.add(MaxPooling2D((2,2), strides=(2,2), name='block2_pool'))

# Block 3
model.add(ZeroPadding2D((1,1)))
model.add(Conv2D(256, (3, 3), activation='relu', name='block3_conv1'))
model.add(ZeroPadding2D((1,1)))
model.add(Conv2D(256, (3, 3), activation='relu', name='block3_conv2'))
model.add(ZeroPadding2D((1,1)))
model.add(Conv2D(256, (3, 3), activation='relu', name='block3_conv3'))
model.add(MaxPooling2D((2,2), strides=(2,2), name='block3_pool'))

# Block 4
model.add(ZeroPadding2D((1,1)))
model.add(Conv2D(512, (3, 3), activation='relu', name='block4_conv1'))
model.add(ZeroPadding2D((1,1)))
model.add(Conv2D(512, (3, 3), activation='relu', name='block4_conv2'))
model.add(ZeroPadding2D((1,1)))
model.add(Conv2D(512, (3, 3), activation='relu', name='block4_conv3'))
model.add(MaxPooling2D((2,2), strides=(2,2), name='block4_pool'))

# Block 5
model.add(ZeroPadding2D((1,1)))
model.add(Conv2D(512, (3, 3), activation='relu', name='block5_conv1'))
model.add(ZeroPadding2D((1,1)))
model.add(Conv2D(512, (3, 3), activation='relu', name='block5_conv2'))
model.add(ZeroPadding2D((1,1)))
model.add(Conv2D(512, (3, 3), activation='relu', name='block5_conv3'))
model.add(MaxPooling2D((2,2), strides=(2,2), name='block5_pool'))

model.add(Flatten())
model.add(Dense(4096, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(4096, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes, activation='softmax'))

In [ ]:
# initiate RMSprop optimizer
opt = keras.optimizers.rmsprop(lr=0.0001, decay=1e-6)
model.compile(loss='categorical_crossentropy',
              optimizer=opt,
              metrics=['accuracy'])

In [ ]:
# data augmentation
datagen = ImageDataGenerator(
    featurewise_center=False,  # set input mean to 0 over the dataset
    samplewise_center=False,  # set each sample mean to 0
    featurewise_std_normalization=False,  # divide inputs by std of the dataset
    samplewise_std_normalization=False,  # divide each input by its std
    zca_whitening=False,  # apply ZCA whitening
    rotation_range=0,  # randomly rotate images in the range (degrees, 0 to 180)
    width_shift_range=0.1,  # randomly shift images horizontally (fraction of total width)
    height_shift_range=0.1,  # randomly shift images vertically (fraction of total height)
    horizontal_flip=True,  # randomly flip images
    vertical_flip=False)  # randomly flip images

# Compute quantities required for feature-wise normalization
# (std, mean, and principal components if ZCA whitening is applied).
datagen.fit(x_train)

In [ ]:
# Plot learning curve on tensor board
tensor_board = TensorBoard(log_dir='/src/tensorboard', batch_size=256)

# Fit the model on the batches generated by datagen.flow().
model.fit_generator(datagen.flow(x_train, y_train, batch_size = 256), 
                    epochs = 100, 
                    validation_data=(x_test, y_test), 
                    workers = 4,
                    callbacks = [tensor_board])

In [ ]:
scores = model.evaluate(x_test, y_test, verbose=1)
print('Test loss:', scores[0])
print('Test accuracy:', scores[1])